# <center><b>MULTIMODAL BIOMETRICS USING GAIT AND EAR FEATURES</b></center>
## <center><u>Done by:</u></center>
- ### K SAI DINESH (CS20B1122)
- ### LOKESH REDDY (CS20B1128)
- ### RUDHRA (CS20B1104)
***

### Import Necessary Libraries

In [40]:
import os
import numpy as np
import cv2
import sys
from scipy.spatial.distance import hamming

### Gait Classification using SIFT Detector

Iterate through each of the test subjects and compare them against the database images.

In [41]:
gait_distances = []
TEST_IMG_DIR = "Test Images/"
test_imgs = os.listdir(TEST_IMG_DIR)
count_i = 1
correct_count = 0

for t_img in test_imgs:

    test_img = cv2.imread(TEST_IMG_DIR + str(t_img))
    
    DATA_IMG_DIR = "Database Images/"
    db_imgs = os.listdir(DATA_IMG_DIR)
    min_dist = sys.maxsize
    count_j = 1
    label = 0
    
    for db_img in db_imgs:

        f_name = DATA_IMG_DIR + db_img
        act_img = cv2.imread(f_name)
        
        gray1 = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(act_img, cv2.COLOR_BGR2GRAY)
        
        sift = cv2.xfeatures2d.SIFT_create()
        
        # Detect blob points using SIFT detector.
        kp1, des1 = sift.detectAndCompute(gray1, None)
        kp2, des2 = sift.detectAndCompute(gray2, None)
        
        # Find matches.
        bf = cv2.BFMatcher()
        matches = bf.match(des1, des2)
        
        # Calculate distance.
        distance = sum([match.distance for match in matches]) / len(matches)
        
        # Updation of distance.
        if(min_dist > distance):
            min_dist = distance
            label = count_j
        
        count_j += 1
    
        gait_distances.append(distance)

    # Matched.
    if(count_i == label):
        correct_count += 1
        
    count_i += 1

print(f"Accuracy : {(correct_count/(len(os.listdir(TEST_IMG_DIR))-1))*100}%")

Accuracy : 45.45454545454545%


### Gait Classification using Hamming Distance

Do the same thing as above but by using Hamming distance as metric.

In [42]:
count_i = 1
correct_count = 0
TEST_IMG_DIR = "Test Images/"

for folder in os.listdir(TEST_IMG_DIR):

    test_img = cv2.imread("Test Images/"+str(folder))
    DATA_IMG_DIR = "Database Images/"
    db_imgs = os.listdir(DATA_IMG_DIR)
    min_dist = sys.maxsize
    count_j = 1
    label = 0

    for db_img in db_imgs:
        f_name = DATA_IMG_DIR + db_img
        act_img = cv2.imread(f_name)
        
        # Convert to gray scale.
        gray1 = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(act_img, cv2.COLOR_BGR2GRAY)

        gray1 = cv2.resize(gray1, (200, 200))
        gray2 = cv2.resize(gray2, (200, 200))

        
        # Flatten them.
        gray1 = gray1.flatten()
        gray2 = gray2.flatten()

        gray1 = list(gray1)
        gray2 = list(gray2)

        # Calculate Hamming Distance.
        distance = hamming(gray1, gray2)
        if(min_dist > distance):
            min_dist = distance
            label = count_j
        
        count_j += 1
    
    # Matched.
    if(count_i == label):
        correct_count += 1
        
    count_i += 1

print(f"Accuracy : {(correct_count/len(os.listdir(TEST_IMG_DIR)))*100}%")

Accuracy : 41.66666666666667%


### Getting the Ear distances

In [43]:
# Step 1: Ear contour detection
def get_ear_contour(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 100, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    ear_contour = max(contours, key=cv2.contourArea)
    return ear_contour

# Step 2: Binarization
def binarize_ear(img, ear_contour):
    mask = np.zeros(img.shape[:2], np.uint8)
    cv2.drawContours(mask, [ear_contour], 0, 255, -1)
    ear_pixels = cv2.bitwise_and(img, img, mask=mask)
    ear_pixels_gray = cv2.cvtColor(ear_pixels, cv2.COLOR_BGR2GRAY)
    _, ear_binary = cv2.threshold(ear_pixels_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return ear_binary

# Step 3: Coordinates normalization
def normalize_ear(ear_binary):
    rows, cols = ear_binary.shape[:2]
    contours, _ = cv2.findContours(ear_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 2:
        ear_contour = contours[1]
    else:
        ear_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(ear_contour)
    norm_ear = ear_binary[y:y + h, x:x + w]
    norm_ear = cv2.resize(norm_ear, (256, 256))
    return norm_ear

# Step 4: Feature extraction (2 steps)
def extract_features(norm_ear):
    # Step 4.1: Get horizontal and vertical projections
    vert_proj = np.sum(norm_ear, axis=0)
    horiz_proj = np.sum(norm_ear, axis=1)

    # Step 4.2: Extract features from projections
    features = []
    for i in range(5):
        x1 = int(i * 20)
        x2 = int((i + 1) * 20)
        vert_segment = vert_proj[x1:x2]
        horiz_segment = horiz_proj[x1:x2]
        vert_mean = np.mean(vert_segment)
        horiz_mean = np.mean(horiz_segment)
        features.append(vert_mean)
        features.append(horiz_mean)
    return features

In [44]:
def Algorithm(IMG_PATH):

    face_cascade = cv2.CascadeClassifier('haarcascade_profileface.xml')
    img = cv2.imread(IMG_PATH)
    img = cv2.flip(img,1)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        img = cv2.flip(img,1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30))


    # Loop over each detected face and extract the ear features
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = img[y:y+h, x:x+w+50]
        # ear_contour = get_ear_contour(face_roi)
        # ear_binary = binarize_ear(face_roi, ear_contour)
        # norm_ear = normalize_ear(ear_binary)
        # ear_features = extract_features(norm_ear)
        # return (ear_features)

        # Convert the face region to grayscale
        gray_face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        
        # Perform edge detection on the grayscale face region
        edges = cv2.Canny(gray_face_roi, 50, 200)
        
        # Apply thresholding to the edge image to extract the ear
        _, thresh = cv2.threshold(edges, 50, 255, cv2.THRESH_BINARY)
        
        # Perform dilation on the thresholded image to fill in gaps
        kernel = np.ones((5,5),np.uint8)
        dilated = cv2.dilate(thresh, kernel, iterations = 1)
        
        # Find contours in the dilated image
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Extract the largest contour as the ear region
        if len(contours) > 0:
            ear_contour = max(contours, key=cv2.contourArea)
            (ex, ey, ew, eh) = cv2.boundingRect(ear_contour)
            ear_img = face_roi[ey:ey+eh, ex:ex+ew]
            
            ear_features = extract_features(ear_img)
            return(ear_features)
        else:
            return (-1)

### Extract Ear features

In [45]:
folder_names = os.listdir("ear_photos/")
db_ear_features = {}
count = 1
for folder in folder_names:
    IMG_FILE_PATH = "ear_photos/"+str(folder)
    db_ear_features[count] = Algorithm(IMG_FILE_PATH)
    count += 1    

c:\Users\hp\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\hp\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [46]:
total_ear_subjects = len(os.listdir("ear_photos/"))
total_ear_subjects

12

### Fill te NULL Values

In [47]:
for i in range(1,total_ear_subjects+1):
    db_ear_features[i] = np.array(db_ear_features[i])
    db_ear_features[i][np.isnan(db_ear_features[i])] = np.nanmean(db_ear_features[i])

### Apply the ear extraction function

In [48]:
test_folder_names = os.listdir("ear_test_photos/")
test_ear_features = {}
count = 1
for folder in test_folder_names:
    IMG_FILE_PATH = "ear_test_photos/"+str(folder)
    test_ear_features[count] = Algorithm(IMG_FILE_PATH)
    count += 1    

### For test ear images to the same as above

In [49]:
for i in range(1,total_ear_subjects+1):
    test_ear_features[i] = np.array(test_ear_features[i])
    test_ear_features[i][np.isnan(test_ear_features[i])] = np.nanmean(test_ear_features[i])

### Get the Ear distances

In [50]:
ear_distances = []
correct_count = 0

for i in range(1,total_ear_subjects+1):

    min_dist = sys.maxsize
    label = 0

    for j in range(1,13):
        curr_dist = abs(np.linalg.norm(test_ear_features[i]-db_ear_features[j]))
        if(curr_dist < min_dist):
            min_dist = curr_dist
            label = j
        ear_distances.append(curr_dist)
    
    if(i == label):
        correct_count += 1

print(f"Accuracy : {(correct_count/len(test_ear_features))*100}%")

Accuracy : 41.66666666666667%


### Combining both Gait and Ear pateerns

### Normalization of distances

In [56]:
gait_distances = gait_distances/np.linalg.norm(gait_distances)
ear_distances = ear_distances/np.linalg.norm(ear_distances)

### Max and Min ranges

In [57]:
print(min(ear_distances),max(ear_distances))
print(min(gait_distances),max(gait_distances))

0.0 0.18320709849192374
0.06084505070314935 0.099773616503489


### Combine ear and gait distances using the weighted sum by using Alpha and Beta values

In [58]:
total_subjects = int(np.sqrt(len(gait_distances)))
alpha = 0
step = 0.01
correct_count = 0
highest_acc = 0

while alpha <= 1:
    beta = 0
    while beta <= 1:
        correct_count = 0
        for i in range(total_subjects):

            start_ind = i*total_subjects
            label = 0
            min_dist = sys.maxsize
            
            for j in range(start_ind, start_ind + total_subjects):
                dist = (alpha * gait_distances[j]) + (beta * ear_distances[j])
                if(min_dist > dist):
                    min_dist = dist
                    label = j - start_ind
            
            if(i == label):
                correct_count += 1
        
        highest_acc = max(highest_acc,correct_count/total_subjects)
        beta += step
    
    alpha += step

print(f"Highest Accuracy : {round(highest_acc*100,2)}%")
# print(f"Accuracy : {round((correct_count/total_subjects)*100,2)}%")

Highest Accuracy : 58.33%


# <center>---End of Notebook---</center>